# Importação de Bibliotecas

In [2]:
#  Importação de bibliotecas completas para experimentos com MLflow e DVC/Dagshub

# Manipulação de dados
import numpy as np
import pandas as pd

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# DVC / Dagshub para versionamento e conexão de dados
import dvc.api
from dagshub.data_engine import datasources
import dagshub

# Rastreamento de experimentos
import mlflow
import mlflow.sklearn
import mlflow.catboost
import mlflow.models.signature
from mlflow.models import infer_signature

# Machine Learning (modelos, treino, métricas)
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error,
    make_scorer
)

# Modelos básicos
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import tempfile
import os
from lightgbm import LGBMClassifier

# Gradient Boosting Libraries
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from xgboost import XGBClassifier
# Utilidades do sistema
import os
import warnings
warnings.filterwarnings('ignore')

print(" Todas as bibliotecas carregadas com sucesso!")


 Todas as bibliotecas carregadas com sucesso!


# Carregamento do Dataset Processado

Nesta etapa, vamos acessar o dataset processado diretamente do repositório versionado no **Dagshub/DVC**.  


In [3]:
# Carrega o dataset processado via Dagshub Data Engine
ds = datasources.get('estrellacouto05/quantum-finance-credit-score', 'processed')

# Exibe as primeiras linhas para verificação
ds.all().dataframe



Accessing as estrellacouto05

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,credit_score_processed.csv,103598542,https://dagshub.com/api/v1/repos/estrellacouto...,text/plain,32908761


Após obter o objeto `datasource`, agora vamos recuperar o link de download do dataset versionado.   
Com essa URL, carregamos os dados diretamente em um **DataFrame Pandas**.

In [4]:
# Obtém os primeiros registros do datasource e extrai o URL de download
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

# Carrega o dataset diretamente a partir do link de download
df = pd.read_csv(dataset_url)

# Exibe as primeiras linhas para verificação
df.head()


Output()

,idade,renda_anual,salario_liquido_mensal,qtd_contas_bancarias,qtd_cartoes_credito,taxa_juros,qtd_emprestimos,dias_atraso_pagamento,qtd_pagamentos_atrasados,variacao_limite_credito,...,tipos_emprestimos_Credit-Builder Loan,tipos_emprestimos_Debt Consolidation Loan,tipos_emprestimos_Home Equity Loan,tipos_emprestimos_Mortgage Loan,tipos_emprestimos_Not Specified,tipos_emprestimos_Payday Loan,tipos_emprestimos_Personal Loan,tipos_emprestimos_Student Loan,tipos_emprestimos_Two or More Types of Loan,score_credito
0,23.0,19114.12,1824.843333,3.0,4,3.0,4.0,3,7.0,11.27,...,False,False,False,False,False,False,False,False,True,2
1,23.0,19114.12,3335.886667,3.0,4,3.0,4.0,1,12.0,11.27,...,False,False,False,False,False,False,False,False,True,2
2,35.0,19114.12,3335.886667,3.0,4,3.0,4.0,3,7.0,9.40,...,False,False,False,False,False,False,False,False,True,2
3,23.0,19114.12,3335.886667,3.0,4,3.0,4.0,5,4.0,6.27,...,False,False,False,False,False,False,False,False,True,2
4,23.0,19114.12,1824.843333,3.0,4,3.0,4.0,6,12.0,11.27,...,False,False,False,False,False,False,False,False,True,2


# Desenvolvimento e experimentos de modelos

## Inicialização do Dagshub com MLflow

In [5]:
# Inicializa o Dagshub com integração ao MLflow
dagshub.init(repo_owner='estrellacouto05',
             repo_name='quantum-finance-credit-score',
             mlflow=True)

print("Dagshub inicializado e MLflow configurado.")


Initialized MLflow to track repo "estrellacouto05/quantum-finance-credit-score"

Repository estrellacouto05/quantum-finance-credit-score initialized!

Dagshub inicializado e MLflow configurado.


In [6]:
# Ativa o registro automático de experimentos com MLflow
mlflow.autolog()

print("MLflow Autolog habilitado.")


2025/08/05 11:05:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/08/05 11:05:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/08/05 11:05:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


MLflow Autolog habilitado.


## Separação de Features e Target



Nesta etapa, vamos separar as variáveis independentes (features) do target (`score_credito`).  
Isso organiza o dataset para os modelos de Machine Learning, que aprenderão a prever o target com base nas features.

In [7]:
# Lista todas as colunas e remove o target 'score_credito' da lista de features
features = list(df.columns)
features.remove("score_credito")

# Exibe o total de features selecionadas
print(f"Total de features: {len(features)}")


Total de features: 48


In [8]:
# Cria os DataFrames para features (X) e target (y)
X = df[features]
y = df["score_credito"]

# Exibe informações básicas para conferência
print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)


Dimensões de X: (100000, 48)
Dimensões de y: (100000,)


## Divisão dos Dados e Escalonamento

Nesta etapa, dividimos os dados em conjuntos de treino e teste para avaliar o desempenho dos modelos de forma justa.  
Utilizamos **70% para treino** e **30% para teste**.

Depois aplicamos o **RobustScaler**, que é ideal para dados com outliers, pois utiliza medianas e intervalos interquartis, evitando distorções.


In [9]:
# Divide os dados em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Conjuntos criados:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


Conjuntos criados:
X_train: (70000, 48), X_test: (30000, 48)
y_train: (70000,), y_test: (30000,)


In [10]:
# Inicializa o RobustScaler
scaler = RobustScaler()

# Ajusta o scaler apenas com os dados de treino e transforma
X_train_scaled = scaler.fit_transform(X_train)

# Aplica a mesma transformação nos dados de teste
X_test_scaled = scaler.transform(X_test)

print("Escalonamento concluído.")
print("Dimensões após escalonamento:", X_train_scaled.shape, X_test_scaled.shape)


Escalonamento concluído.
Dimensões após escalonamento: (70000, 48) (30000, 48)


## Função de Avaliação e Log no MLflow (Classificação)

- **Métricas principais:** precision, recall e f1-score para cada classe.
- **Destaque do recall da classe 0 (Poor)**, que é a mais importante.
- **Matriz de confusão** gerada como gráfico e registrada no MLflow como artefato.
- **Registro do modelo** no MLflow (CatBoost, XGBoost, LightGBM ou sklearn).

In [11]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    # Faz previsões
    predictions = model.predict(X_test)

    # Calcula métricas de classificação
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, labels=[0,1,2], zero_division=0)
    report = classification_report(y_test, predictions, digits=3)

    # Log de métricas macro (média das classes)
    mlflow.log_metric("Precision_macro", precision.mean())
    mlflow.log_metric("Recall_macro", recall.mean())
    mlflow.log_metric("F1_macro", f1.mean())

    # Log detalhado de métricas por classe
    mlflow.log_metric("Recall_class_0_Poor", recall[0])
    mlflow.log_metric("Precision_class_0_Poor", precision[0])
    mlflow.log_metric("F1_class_0_Poor", f1[0])
    mlflow.log_metric("Recall_class_1_Standard", recall[1])
    mlflow.log_metric("Recall_class_2_Good", recall[2])

    # Cria assinatura do modelo para salvar no MLflow
    signature = infer_signature(X_test, predictions)

    # Salva o modelo de acordo com o tipo
    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    # Gera matriz de confusão
    cm = confusion_matrix(y_test, predictions, labels=[0,1,2])
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Poor', 'Standard', 'Good'], yticklabels=['Poor', 'Standard', 'Good'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title(f'Matriz de Confusão - {model_name}')

    # Salva a figura temporariamente e envia como artefato para o MLflow
    with tempfile.TemporaryDirectory() as tmpdir:
        cm_path = os.path.join(tmpdir, "confusion_matrix.png")
        plt.savefig(cm_path)
        plt.close()
        mlflow.log_artifact(cm_path, artifact_path="confusion_matrix")

    # Print no console
    print(f"=== Avaliação do Modelo: {model_name} ===")
    print(report)
    print(f"Recall da classe 0 (Poor): {recall[0]:.3f}")

## Etapa de Treinamento de Modelos – XGBoost

A partir deste ponto, iniciamos o ciclo de **treinamento de modelos com MLflow**.  
Começamos com o **XGBoost Classifier**, um modelo de boosting altamente eficaz em dados tabulares.

- Foi usado **RandomizedSearchCV** com 30 combinações aleatórias de hiperparâmetros.
- Todas as métricas (precision, recall, f1-score) e a matriz de confusão foram registradas no MLflow.
- Nosso foco principal é **maximizar o recall da classe 0 (Poor)**, já que é a categoria mais crítica para o projeto.

Após avaliar o XGBoost, avançaremos para **LightGBM** e **CatBoost** para comparar os resultados.


In [26]:
with mlflow.start_run(run_name="XGBoost_Classifier_RandomSearch"):

    # Definindo a grade de hiperparâmetros ampliada
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'min_child_weight': [1, 3, 5]
    }

    # Modelo base XGBoost
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

    # RandomizedSearchCV para explorar combinações de hiperparâmetros
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_distributions,
        n_iter=30,                # número de combinações aleatórias a testar
        scoring='recall_macro',   # métrica principal
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina com os dados escalonados
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores parâmetros no MLflow
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo
    evaluate_and_log_model("xgboost", "XGBoost Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/03 22:02:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits


2025/08/03 22:13:25 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: XGBoost Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.767     0.727     0.747      8805
           1      0.763     0.795     0.779     15873
           2      0.669     0.642     0.655      5322

    accuracy                          0.748     30000
   macro avg      0.733     0.721     0.727     30000
weighted avg      0.747     0.748     0.747     30000

Recall da classe 0 (Poor): 0.727
🏃 View run XGBoost_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f9ec195a72dd4afb9b91f99ef5f727b2
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## Treinamento do Modelo LightGBM com MLflow

Agora vamos treinar o **LightGBM Classifier**, outro modelo de boosting eficiente e mais rápido que o XGBoost em muitos cenários.

- Usaremos **RandomizedSearchCV** com 30 combinações, igual ao XGBoost, para manter o tempo de treino semelhante (~150 fits).
- Vamos incluir parâmetros específicos do LightGBM, como **num_leaves**, que controla a complexidade das árvores.
- Métrica de busca: **recall_macro**, priorizando melhor cobertura da classe “Poor”.
- Tudo será registrado no **MLflow** para comparação posterior.


In [28]:
with mlflow.start_run(run_name="LightGBM_Classifier_RandomSearch"):

    # Definição do espaço de busca
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9, -1],  # -1 = sem limite de profundidade
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [15, 31, 63, 127],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0]
    }

    # Modelo base LightGBM
    lgb = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    # RandomizedSearchCV – 30 combinações x 5 folds = 150 fits
    random_search = RandomizedSearchCV(
        estimator=lgb,
        param_distributions=param_distributions,
        n_iter=30,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina o modelo
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM Classifier RandomSearch", best_model, X_test_scaled, y_test)


2025/08/03 22:21:43 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010766 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/03 22:30:14 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: LightGBM Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.781     0.778     0.779      8805
           1      0.788     0.810     0.799     15873
           2      0.748     0.692     0.719      5322

    accuracy                          0.780     30000
   macro avg      0.773     0.760     0.766     30000
weighted avg      0.779     0.780     0.779     30000

Recall da classe 0 (Poor): 0.778
🏃 View run LightGBM_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eaf41b22355a40d9a6a6b939818b364d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Melhor foi light gbm 

## Experimento: Random Forest Classifier com RandomizedSearchCV

Neste experimento, aplicamos o modelo `RandomForestClassifier` para classificação dos scores de crédito, utilizando `RandomizedSearchCV` com validação cruzada (CV=5) para otimização de hiperparâmetros.

A escolha pelo `Random Forest` visa avaliar o desempenho de um modelo de ensemble tradicional, mais interpretável, como alternativa aos métodos baseados em Gradient Boosting (XGBoost e LightGBM).

**Configurações do experimento:**
- 50 combinações testadas com `RandomizedSearchCV` (`n_iter=50`)
- Métrica de avaliação principal: `recall_macro`
- Hiperparâmetros otimizados: número de estimadores, profundidade máxima (com limite para evitar overfitting), amostragem mínima para splits e folhas, tipo de critério, entre outros.
- Avaliação com função customizada `evaluate_and_log_model`, registrando métricas detalhadas (precision, recall, f1-score, matriz de confusão), com foco no `recall` da classe 0 (`Poor`), além de logging via MLflow.

O objetivo é verificar se o Random Forest entrega resultados competitivos e interpretar seu comportamento em comparação com os modelos anteriores.


In [ ]:
# Início do experimento com MLflow
with mlflow.start_run(run_name="RandomForest_Classifier_RandomSearch"):

    # Instancia o modelo base
    rf = RandomForestClassifier(random_state=42)

    # Espaço de busca ajustado (sem max_depth=None para evitar overfitting)
    param_distributions = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy']
    }

    # RandomizedSearchCV com 50 combinações
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treinamento
    random_search.fit(X_train_scaled, y_train)

    # Modelo final
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avaliação + log via função customizada
    evaluate_and_log_model("sklearn", "RandomForest Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/05 11:25:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025/08/05 12:00:24 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


🏃 View run auspicious-deer-16 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/702b80c4c87b4456a148eb28e2e1b791
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run monumental-ox-450 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/83a8a9a3b726467fa8b2980745aa22ca
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run grandiose-zebra-368 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/8f247204dc2d4f8a9a0ff3d2547aa025
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run burly-ape-995 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e8bf5ee20de9424099f1d17821de5018
🧪 View experiment at: https://dagsh

=== Avaliação do Modelo: RandomForest Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.780     0.754     0.767      8805
           1      0.778     0.807     0.793     15873
           2      0.713     0.673     0.693      5322

    accuracy                          0.768     30000
   macro avg      0.757     0.745     0.751     30000
weighted avg      0.767     0.768     0.767     30000

Recall da classe 0 (Poor): 0.754
🏃 View run RandomForest_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/df711845800e452aa800d9578ce58457
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run enchanting-shad-321 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ab7344499f74817a85e876013514d9f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run serious-asp-250 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c796d96f67054d22871e19825ce2175d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run invincible-rook-21 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/22ef267fe2ca4032a2ddcce5d44abc9a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run angry-asp-955 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7d9fc16a65f342b798a7e2299bdbfabd
🧪 View experiment at: https://dagshub

🏃 View run worried-steed-201 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/09b6d6b37e724f3a8a1da73fa71fc074
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-mare-104 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ff41165c2554efe88c2927706ff5571
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-smelt-845 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e221b0dce6884322ad9a016113666026
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run big-asp-866 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7bead2d39d444bf188e4735a0c8630a6
🧪 View experiment at: https://dagshub.com/est

## Fine-Tuning do LightGBM com RandomizedSearchCV

Nesta etapa, realizamos um ajuste fino (fine-tuning) no modelo LightGBM com foco em melhorar o **recall da classe 0 (Poor)**.  
Baseado nos melhores parâmetros anteriores, restringimos os ranges de busca para explorar variações próximas e mais promissoras.  
Usamos `RandomizedSearchCV` com `n_iter=50` e validação cruzada (`cv=5`), mantendo o controle de overfitting por meio de `num_leaves`, `subsample` e `colsample_bytree`.  
Os resultados serão registrados automaticamente no MLflow via integração com o Dagshub.


In [13]:
with mlflow.start_run(run_name="LightGBM_FineTuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [300, 400, 500],
        'learning_rate': [0.05, 0.08, 0.1],
        'num_leaves': [95, 127, 150],
        'max_depth': [5, 7, 9, -1],
        'subsample': [0.6, 0.7, 0.8],
        'colsample_bytree': [0.7, 0.85, 1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Loga os melhores hiperparâmetros
    mlflow.log_params(randomized_search.best_params_)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM FineTuned", best_model, X_test_scaled, y_test)

2025/08/05 12:35:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 13:25:15 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


=== Avaliação do Modelo: LightGBM FineTuned ===
              precision    recall  f1-score   support

           0      0.785     0.786     0.785      8805
           1      0.793     0.812     0.803     15873
           2      0.755     0.701     0.727      5322

    accuracy                          0.784     30000
   macro avg      0.778     0.766     0.772     30000
weighted avg      0.784     0.784     0.784     30000

Recall da classe 0 (Poor): 0.786
🏃 View run LightGBM_FineTuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/05cfe5fa3fa544bbba01799cb7f7b903
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Fine-tuning adicional no LightGBM com foco em `n_estimators` e `learning_rate`

Neste experimento, mantemos os melhores hiperparâmetros encontrados anteriormente (como `num_leaves`, `max_depth`, `subsample` e `colsample_bytree`) e realizamos um ajuste fino especificamente sobre os parâmetros `n_estimators` e `learning_rate`.

O objetivo é verificar se o aumento do número de estimadores (de 500 para 750 e 1000) combinado com taxas de aprendizado mais baixas (0.05, 0.03 e 0.01) resulta em ganho de recall, principalmente da classe 0 (Poor), sem causar overfitting.

A técnica continua sendo o `RandomizedSearchCV` com 50 iterações e validação cruzada (cv=5), mantendo o padrão adotado em experimentos anteriores.



In [14]:
with mlflow.start_run(run_name="LightGBM_Estimators_LearningRate_Tuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [500, 750, 1000],
        'learning_rate': [0.05, 0.03, 0.01],
        'num_leaves': [127],
        'max_depth': [-1],
        'subsample': [0.7],
        'colsample_bytree': [1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Log dos principais hiperparâmetros selecionados
    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)

    # Avaliação e log do modelo
    evaluate_and_log_model("lightgbm", "LightGBM Estimators-LR Tuning", best_model, X_test_scaled, y_test)

2025/08/05 13:57:07 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 14:20:52 INFO mlflow.sklearn.utils: Logging the 5 best runs, 4 runs will be omitted.


=== Avaliação do Modelo: LightGBM Estimators-LR Tuning ===
              precision    recall  f1-score   support

           0      0.785     0.789     0.787      8805
           1      0.794     0.812     0.803     15873
           2      0.759     0.701     0.729      5322

    accuracy                          0.786     30000
   macro avg      0.779     0.767     0.773     30000
weighted avg      0.785     0.786     0.785     30000

Recall da classe 0 (Poor): 0.789
🏃 View run LightGBM_Estimators_LearningRate_Tuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eb3c5dfe89f44412971d50969d0c882f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


###  Escolha final do modelo LightGBM (Fine-Tuning)

Após dois ciclos de fine-tuning com o LightGBM, foram comparadas duas configurações principais:

| Modelo | `n_estimators` | `learning_rate` | Recall Classe 0 (Poor) |
|--------|----------------|------------------|--------------------------|
| A (1º Fine-tune) | **500** | **0.1** | **0.786** |
| B (2º Fine-tune) | **1000** | **0.05** | **0.789** |

Apesar do modelo B utilizar o dobro de árvores com um `learning_rate` mais baixo (0.05), **a melhora no recall da classe 0 foi mínima** (de 0.786 para 0.789), sem ganho significativo em outras métricas. Além disso, esse aumento nos estimadores **eleva o custo computacional e o risco de overfitting** sem retorno proporcional em desempenho.

####  Conclusão:
Optamos por manter o modelo com:
- `n_estimators = 500`
- `learning_rate = 0.1`

Essa combinação mostrou-se mais eficiente, com ótimo desempenho geral, menor complexidade e melhor equilíbrio entre performance e custo de treinamento.




# Registro de Modelo em Produção

Após treinar e avaliar o modelo, podemos registrá-lo oficialmente no **Model Registry do MLflow**.  
Isso permite versionar o modelo, promovê-lo para produção e gerenciar futuras atualizações.  

- Usamos o **run_id** obtido no link do MLflow (na interface Dagshub).
- Escolhemos um nome amigável e consistente para o modelo, neste caso: `credit-score-model`.

In [30]:
# Define o run_id do experimento que você deseja registrar
run_id = "eaf41b22355a40d9a6a6b939818b364d"

# Registra o modelo no MLflow Model Registry
mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name="credit-score-model"
)

print(f"✅ Modelo registrado como 'credit-score-model' (run_id: {run_id})")


Successfully registered model 'credit-score-model'.
2025/08/03 22:48:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit-score-model, version 1
Created version '1' of model 'credit-score-model'.


✅ Modelo registrado como 'credit-score-model' (run_id: eaf41b22355a40d9a6a6b939818b364d)
